## [00] Project Planning

### Objective

Build an LLM-powered **Schema Drift Detection & Auto‑Resolution Agent** in Databricks. The goal is to **automatically identify** structural changes in Delta table schemas and **suggest code updates** (PySpark or SQL) to handle them. This helps avoid broken pipelines and reduces manual intervention.

### Problem Statement

- **Schema drift** occurs when data schemas change unexpectedly (e.g., added, removed, renamed columns, or changed data types).
- These changes can **break ETL pipelines**, cause downstream failures, and require manual fixes.
- We need a system that can **detect drift**, **explain it to humans**, and **generate resolution code** automatically within Databricks.

### High Level Architecture

1. **Fetch** current and new table schemas from Delta.
2. **Compare** schemas to compute a structured diff.
3. **Generate prompt** for LLM, describing the schema changes.
4. **LLM returns** a code snippet for resolution (e.g., modify DataFrame or ALTER TABLE).
5. **Display** the suggestion in notebook; allow engineer to **review and execute**.
6. *(Future scope)* Automatically apply changes or log outputs for monitoring.

### Milestones Overview

| Week | Main Focus |
|------|------------|
| 1    | Research, planning, schema evolution tests |
| 2    | Schema comparison logic (PySpark) |
| 3    | Prompt engineering + LLM integration |
| 4    | End-to-end prototype development |
| 5    | Testing, validation, prompt refinement |
| 6    | Documentation, demo, final wrap-up |

### Learning: Delta Schema Enforcement & Evolution
**Link**: https://www.databricks.com/blog/2019/09/24/diving-into-delta-lake-schema-enforcement-evolution.html

#### 1. Schema Enforcement Definition and Behavior

Delta Lake’s schema enforcement (aka schema validation) ensures data quality by rejecting any writes that don’t match the expected schema of a target table. It acts like a gatekeeper, blocking writes with extra columns or mismatched types

#### 2. Enforcement Rules

The blog outlines these key rules for schema enforcement:
- **No additional columns** in the incoming DataFrame that aren't already present in the table.

- **Missing columns** are allowed and filled with ```NULL```.

- **Data types must match exactly** - strict type matching is enforced.

- **Column names must match case-insensitively**, and Delta Lake forbids having both ```Foo``` and ```foo``` as seperate columns.

#### 3. Schema Evolution Mechanisms
When ```.option("mergeSchema", "true")``` is used:
  - New columns added in the DataFrame are automatically appended to the table’s schema.

  - Nested struct fields are also supported.

  - Type changes from nullable to another type (e.g. ```NullType``` → ```StringType```) and certain upcasts (```Byte → Short → Integer```) are handled.
  This makes the evolution seamless for common changes.

#### 4. Use Cases & Trade-offs
- **Schema enforcement** is ideal for production-quality tables feeding downstream systems like ML models and BI dashboards—offering strong data integrity and preventing accidental schema drift.

- **Schema evolution** is useful when you intend to change schemas, letting you add columns without manual intervention. However:
  - It doesn’t handle **column removal**, **in-place type changes**, or **renames** (especially case changes)—these require ```.option("overwriteSchema", "true")``` or DDL commands.
  - It’s purposely limited so as not to silently break downstream expectations.

### Learning: Auto Loader Schema Drift
**Link:** https://community.databricks.com/t5/technical-blog/schema-management-and-drift-scenarios-via-databricks-auto-loader/ba-p/63393

#### 1. Schema Inference Mechanism
- Auto Loader **samples up to 50 GB or 1,000 files** to infer the schema for the input directory.  

- It writes inferred schemas into a ```_schemas``` folder under the configured ```cloudFiles.schemaLocation```—this becomes your source of truth for schema evolution over time

#### 2. Supported File Formats & Type Inference
- **JSON, CSV, XML:** Auto Loader infers everything as strings unless you enable ```.option("cloudFiles.inferColumnTypes", "true")```.  

- **Text, Binary:** These formats don't support evolution.

- **Parquet, Avro:** Typed formats retain their native data types and are merged during sampling. On type conflicts, it chooses the widest type (e.g., long over int), unless overridden by ```schemaHints```.

#### 3. Automatic Schema Evolution  
- When Auto Loader detects **new columns**, it:
  1. Stops the stream with an `UnknownFieldException`.
  2. Merges the new column(s) into the schema, placing them at the end.
  3. Keeps existing column types intact.  
- **Note:** Instructs a pipeline restart (e.g., via Lakeflow Jobs) to resume with updated schema.

#### 4. ```cloudFiles.schemaEvolutionMode``` Control  
Auto Loader supports several modes:

| Mode | Behavior |
|------|----------|
| `addNewColumns` | (Default) Stream **fails**, schema file updated to include new columns. |
| `rescue` | Stream continues; unexpected fields go into a `_rescued_data` column. |
| `failOnNewColumns` | Stream fails; schema is NOT updated until manually changed. |
| `none` | Stream continues; new columns are ignored unless `rescuedDataColumn` is set. |

- The default behavior is `addNewColumns` unless a user-supplied schema is provided, in which case the default changes to `none`.

#### 5. `_rescued_data` Column  
- If using ```rescue``` mode (or enabling ```rescuedDataColumn```), unmatched fields are captured in a ```_rescued_data``` column rather than being dropped.  
- You can rename this column via the ```rescuedDataColumn``` option.

#### 6. Partition Columns Are Ignored in Drift  
- Auto Loader can detect Hive-style partition columns (e.g., ```/date=2025-01-01/```).  
- **Partition evolution is not supported**: new partitions will not be added to the schema unless manually specified using ```cloudFiles.partitionColumns```.

### How Research Informs the Project

#### 1. Schema Enforcement & Evolution (Delta Lake)
- Clarifies which schema changes Delta *handles automatically*—such as column additions, nullable upcasts, and nested fields—ensuring the agent avoids duplicated work.

- Defines the boundaries of *unsupported changes* (e.g., column drops, renames, type conversions) that require explicit intervention, allowing the agent to detect and prompt for these specific adjustments.

- Supports development of logic that differentiates between "safe" drift (handled by system) and "risky" drift (requiring agent-generated code), improving system reliability

#### 2. Auto Loader Schema Management (Streaming Context)
- Highlights how Auto Loader samples schemas and manages drift through ```_schemas``` metadata and ```cloudFiles.schemaEvolutionMode``` configurations (e.g., ```addNewColumns```, ```rescue```), which enables the agent to align its behavior with real-time ingestion pipelines.

- Identifies scenarios—like data captured in ```_rescued_data```, unsupported partition-schema changes, or streaming failures—where the agent should suggest specific modes or extraction logic, enhancing streaming robustness.

### Schema Drift Types to Detect

We categorize schema drift types into the following categories:

1. **Column Added**
    - A new column appears in the updated schema.
    - Example: ```age``` (IntegerType) added.

2. **Column Removed**
    - A column present in the old schema is missing in the new one.
    - Example: ```name``` (StringType) removed.

3. **Column Type Changed**
    - A column exists in both schemas, but its data type is different.
    - Example: ```id```: IntegerType → LongType

Note: Complex changes like **nested fields**, **renamed columns**, or **reordered fields** are not handled in Week 1.

### Design Plan for Schema Drift Agent

The Schema Drift Agent will consist of:

1. **Schema Comparison Engine**
   - Written in Python (`schema_diff.py`)
   - Takes two JSON schemas: old and new
   - Uses `load_schema()` to read them, and `compute_schema_diff()` to compare them
   - Produces a list of changes like:
     - Column X added
     - Column Y removed
     - Column Z changed type
     - Column X was renamed to A

2. **Prompt Generation Logic**
   - Based on schema diffs, we will construct prompts for an LLM
   - Example: “Column `age` added. Please provide PySpark code to handle this.”

3. **Test Schema Setup**
   - We simulate schema drift using files like:
     - `schema_test_old.json`
     - `schema_test_new.json`
   - These files contain simplified versions of table schemas for easy validation

### Rename Schema Drift Detection

#### Research & Insights

- **Estuary (https://estuary.dev/blog/schema-drift/)** underscores that real-world data pipelines must detect structural changes—like renames—early, because upstream variance can break downstream systems.
- **Metaplane (https://www.metaplane.dev/blog/three-ways-to-track-schema-drift-in-snowflake)** recommends frequent schema snapshot comparison to catch drift types including renames.
- **Snowflake/Matia Guide (https://www.matia.io/blog/how-to-build-a-schema-drift-alert-system)** highlights renames as a critical class of schema drift, and suggests treating them as a combination of removal and addition unless intelligently paired.

**Key Takeaway:**  
To detect renames, we must map removed → added column pairs, ideally using heuristics (name similarity + type matching), with optional manual hints.

#### Design Approach

**Objective**: Extend ```compute_schema_diff()``` to mark and report renamed columns.

1. **Extend ```diff``` result structure**:
   - Add a ```renamed``` list alongside ```added```, ```removed```, ```type_changed```.

2. **Develop ```detect_renames(diff, old_fields, new_fields, hints=None)```**:
   - **Manual override**: use ```rename_hints.json``` map (e.g., ```"fullname": "name"```).
   - **Automatic heuristic**: match removed and added names by similarity (e.g., ```startswith```, levenshtein) **and** require same type.
   - If renamed → add ```{"from": old, "to": new}```, and remove from ```added```/```removed```.

3. **Integrate detection**:
   - Run ```detect_renames()``` after computing the flat/nested diff, before returning result.

#### Update (17 July 2025): Heuristic Rename Detection Removed

I have removed **heuristic-based rename detection** from the schema drift agent due to inconsistent behavior and unreliable matches in edge cases. While the approach attempted to pair removed and added fields using name similarity (e.g., ```startswith``` logic), it led to:
- False positives for unrelated fields with similar names
- Missed matches when nested paths or casing differences were involved
- Complexity in test coverage and debugging

**Current Strategy:**  
I now rely **only on manual rename hints** (```rename_hints``` dictionary) to detect renamed columns. These mappings explicitly specify expected renames (e.g., ```"created_at" → "account_created"```) and allow for clear, controlled detection. If the renamed field’s type also changes, that is captured in the ```type_changed``` section.

This change simplifies logic, improves precision, and ensures the agent aligns with real-world expectations in production-grade data platforms.

### Why Reordered Column Drift Is Not Detected

Explored whether to include **field reordering** (e.g., changing column order from ```["id", "name"]``` to ```["name", "id"]```) as a schema drift type.

#### Decision:
Reordering is **not treated as drift**, and will not be detected or resolved by the agent.

#### Reasoning:
- **Delta Lake & Spark**: Column order is not enforced; schema validation only checks for name and type matches.
- **Parquet/Avro formats**: Internally store columnar data by name, not order.
- **SQL Engines & BI Tools**: Reference columns by name — order does not affect execution.
- **Reordering does not break pipelines**, transformations, or queries in modern systems.

#### Exceptions:
- CSV-based workflows (order-sensitive by design)
- Scripts using positional access like ```df.iloc[:, 0]``` (bad practice)
- Legacy fixed-position APIs (rare)

Because these cases are uncommon in modern data platforms like Databricks, we classify reordering as **non-breaking drift**.

It may be flagged visually by users, but **no automated resolution or LLM prompt is required**.

### Rename + Type Change Handling (Design Update)

#### Problem

My initial `detect_renames()` implementation only recognized renamed columns if their data types **remained unchanged**. As a result, important drifts like:

- `created_at` → `account_created` (`TimestampType` → `DateType`)
- `last_login` → `lastLogin` (`TimestampType` → `StringType`)

were not marked as renames, and their type changes were also missed.

This led to incorrect or incomplete drift reports and caused several test assertions to fail.

#### Root Cause

The detection logic skipped rename matches when: 
```if old_fields_map.get(old_name) == new_fields_map.get(new_name):```

This condition only allowed renames where the types matched exactly, ignoring valid cases where both the name changed and type evolved.

#### Solution
I updated the ```detect_renames()``` function to:

Allow renames even if types differ

Add an entry to ```type_changed``` when a renamed field’s type also changes

Apply this logic to both manual rename hints and heuristic rename matches

I also added a ```flatten_fields()``` utility to recursively support nested structs, so renames and type changes are detected for fields like ```metadata.score``` and ```preferences.notifications```.

### Future Scope: Struct-Level Rename Detection

Currently, the schema drift agent detects renamed fields using **fully qualified field names** (e.g., ```metadata.source``` → ```metadata.origin```) and relies on **explicit rename hints** for each renamed field.

However, it does **not support renames at the struct level** (e.g., ```metadata``` → ```meta_info```) unless all nested fields are individually mapped in ```rename_hints```. This can lead to false positives like:

- ```"metadata.source"``` showing up as **removed**
- ```"meta_info.source"``` showing up as **added**

even though the structure and types may be identical.

**Planned enhancement:**  
Support intelligent struct-level rename detection by:
- Matching renamed parent structs based on field overlap and type similarity
- Automatically resolving child field renames when parent structs are identified as renamed
- Reducing the need for manually listing every nested rename in ```rename_hints```

This feature would improve drift detection for large or deeply nested schemas and make the agent more scalable and intelligent.